In [1]:
# Importing the libraries

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

# Ignoring harless warnings

import warnings
warnings.filterwarnings("ignore")

import pandasql as psql

In [2]:
# Load the Defult Credit Card cilents data

dccdata = pd.read_csv(r"D:\iiit notes\Programming\AI\Internship practice\44 season 17-jul-2021\Default_Credit_Card_Clients_SVM.csv",header = 0)
dccdata.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [3]:
dccdata.rename({'default payment next month' : 'CC_Default'}, axis = 1, inplace=True)
dccdata.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,CC_Default
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [4]:
# Count the target or dependent variable by '0' or '1' and
# their proportion (> 10: 1, then the dataset is imbalance dataset

dccdata_count = dccdata.CC_Default.value_counts()
print('Class 0: ', dccdata_count[0])
print('Class 1: ', dccdata_count[1])
print('Proportion: ', round(dccdata_count[0]/dccdata_count[1], 2), ': 1')
print('Total CC Trans: ', len(dccdata))   

Class 0:  23364
Class 1:  6636
Proportion:  3.52 : 1
Total CC Trans:  30000


In [5]:
# Missing value data size

dccdata_missing = (dccdata[(dccdata.EDUCATION == 0) | (dccdata.MARRIAGE == 0)])
dccdata_missing.shape

(68, 25)

In [6]:
dccdata_no_missing = dccdata[(dccdata.EDUCATION != 0) &  (dccdata.MARRIAGE != 0)]
dccdata_no_missing.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,CC_Default
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [7]:
dccdata1 = dccdata_no_missing.copy()
dccdata1.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,CC_Default
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [8]:
# Converting into dummies

dccdata1 = pd.get_dummies(dccdata1, columns=['SEX','EDUCATION','MARRIAGE','PAY_1','PAY_2',
                                            'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'])
dccdata1.head()

,ID,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,...,PAY_6_-2,PAY_6_-1,PAY_6_0,PAY_6_2,PAY_6_3,PAY_6_4,PAY_6_5,PAY_6_6,PAY_6_7,PAY_6_8
0,1,20000,24,3913,3102,689,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,2,120000,26,2682,1725,2682,3272,3455,3261,0,...,0,0,0,1,0,0,0,0,0,0
2,3,90000,34,29239,14027,13559,14331,14948,15549,1518,...,0,0,1,0,0,0,0,0,0,0
3,4,50000,37,46990,48233,49291,28314,28959,29547,2000,...,0,0,1,0,0,0,0,0,0,0
4,5,50000,57,8617,5670,35835,20940,19146,19131,2000,...,0,0,1,0,0,0,0,0,0,0


In [9]:
cols = ['LIMIT_BAL','AGE','BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

In [10]:
# Identify independent and target variable

x = dccdata1.drop(['CC_Default','ID'], axis=1)
y = dccdata1['CC_Default']

In [11]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state = 42)
x_test_f1 = x_test.copy()

In [12]:
# Scaling the features by using MinMaxScaler

from sklearn.preprocessing import MinMaxScaler

mmscaler = MinMaxScaler(feature_range=(0, 1))

x_train[cols] = mmscaler.fit_transform(x_train[cols])
x_train = pd.DataFrame(x_train)

x_test[cols] = mmscaler.fit_transform(x_test[cols])
x_test = pd.DataFrame(x_test)

# Random Forest without PCA

In [13]:
# Build Random Forest classification model and Train the model using the training sets

from sklearn.ensemble import RandomForestClassifier

dccdataRF = RandomForestClassifier(bootstrap=True,ccp_alpha=0.0, class_weight=None,
                                  criterion='entropy', max_depth=None, max_features='auto',
                                  max_leaf_nodes=None, max_samples=None,
                                  min_impurity_decrease=0.0, min_impurity_split=None,
                                  min_samples_leaf=1, min_samples_split=2,
                                  n_jobs=None, oob_score=False, random_state=0, verbose=0,
                                  warm_start=False,min_weight_fraction_leaf=0.0, n_estimators=500)

dccdataRF = dccdataRF.fit(x_train, y_train)

# Predict the model with test data set

y1_pred = dccdataRF.predict(x_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y1_pred))
print(classification_report(y_test, y1_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y1_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y1_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y1_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y1_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve 
print('roc_auc_score:', round(roc_auc_score(y_test, y1_pred), 3))

[[6721  291]
 [1342  626]]
              precision    recall  f1-score   support

           0       0.83      0.96      0.89      7012
           1       0.68      0.32      0.43      1968

    accuracy                           0.82      8980
   macro avg       0.76      0.64      0.66      8980
weighted avg       0.80      0.82      0.79      8980

Accuracy: 81.82 %
Precision: 81.82 %
Recall: 81.82 %
f1-score: 81.82 %
roc_auc_score: 0.638


# Decision Tree without PCA

In [14]:
# To build the decision tree model with Over sampling 

from sklearn.tree import DecisionTreeClassifier 

dccdataDT = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                   max_depth=None, max_features=None, max_leaf_nodes=None,
                                   min_impurity_decrease=0.0, min_impurity_split=None,
                                   min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0,
                                   random_state=None, splitter='best')

dccdataDT = dccdataDT.fit(x_train,y_train)

# Predict with test data

y3_pred = dccdataDT.predict(x_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y3_pred))
print(classification_report(y_test, y3_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y3_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y3_pred) * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y3_pred) * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y3_pred) * 100, 2)), '%')

# Area under ROC curve 
print('roc_auc_score:', round(roc_auc_score(y_test, y3_pred), 3))

[[3230 3782]
 [ 835 1133]]
              precision    recall  f1-score   support

           0       0.79      0.46      0.58      7012
           1       0.23      0.58      0.33      1968

    accuracy                           0.49      8980
   macro avg       0.51      0.52      0.46      8980
weighted avg       0.67      0.49      0.53      8980

Accuracy: 48.59 %
Precision: 23.05 %
Recall: 57.57 %
f1-score: 32.92 %
roc_auc_score: 0.518


# Logistic Regression without PCA

In [15]:
# To build the 'Logistic Regression' model with random sampling

from sklearn.linear_model import LogisticRegression

dccdataLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                               intercept_scaling=1, max_iter=100, multi_class='auto', 
                               n_jobs=None, penalty='l2', random_state=None,
                               solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

dccdataLR = dccdataLR.fit(x_train,y_train)

# Predict the model with test data set

y5_pred = dccdataLR.predict(x_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y5_pred))
print(classification_report(y_test, y5_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y5_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y5_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y5_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y5_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve 
print('roc_auc_score:', round(roc_auc_score(y_test, y5_pred), 3))

[[6652  360]
 [1255  713]]
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      7012
           1       0.66      0.36      0.47      1968

    accuracy                           0.82      8980
   macro avg       0.75      0.66      0.68      8980
weighted avg       0.80      0.82      0.80      8980

Accuracy: 82.02 %
Precision: 82.02 %
Recall: 82.02 %
f1-score: 82.02 %
roc_auc_score: 0.655


# KNN without PCA

In [16]:
# Initialize an array that stores the Accuracy

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import sklearn.metrics as metrics
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.metrics import classification_report, confusion_matrix

accuracy = []

for a in range(1, 10, 1):
    k = a
    dccdata1KNN = KNeighborsClassifier(n_neighbors=k)
    dccdata1KNN.fit(x_train, y_train)
    y6_pred = dccdata1KNN.predict(x_test)
    print('KNN_K_value = ', a)
    print("Accuracy:", (round(metrics.accuracy_score(y_test, y6_pred) * 100, 2)), "%")
    print("Precision:", (round(metrics.precision_score(y_test, y6_pred, average='micro') * 100, 2)), '%')
    print("Recall:", (round(metrics.recall_score(y_test, y6_pred, average='micro') * 100, 2)), "%")
    print("f1-score:", (round(metrics.f1_score(y_test, y6_pred, average='micro') * 100, 2)), '%')
    print('roc_auc_score:', round(roc_auc_score(y_test, y6_pred), 3))
    print(confusion_matrix(y_test, y6_pred))
    print('----------------------------------------------------------------------------------------------')

KNN_K_value =  1
Accuracy: 72.22 %
Precision: 72.22 %
Recall: 72.22 %
f1-score: 72.22 %
roc_auc_score: 0.599
[[5739 1273]
 [1222  746]]
----------------------------------------------------------------------------------------------
KNN_K_value =  2
Accuracy: 78.72 %
Precision: 78.72 %
Recall: 78.72 %
f1-score: 78.72 %
roc_auc_score: 0.58
[[6654  358]
 [1553  415]]
----------------------------------------------------------------------------------------------
KNN_K_value =  3
Accuracy: 77.36 %
Precision: 77.36 %
Recall: 77.36 %
f1-score: 77.36 %
roc_auc_score: 0.629
[[6217  795]
 [1238  730]]
----------------------------------------------------------------------------------------------
KNN_K_value =  4
Accuracy: 80.07 %
Precision: 80.07 %
Recall: 80.07 %
f1-score: 80.07 %
roc_auc_score: 0.609
[[6663  349]
 [1441  527]]
----------------------------------------------------------------------------------------------
KNN_K_value =  5
Accuracy: 79.51 %
Precision: 79.51 %
Recall: 79.51 %
f1-scor

# SVM-Gaussian without PCA

In [17]:
# Training the SVM alforithm

from sklearn.svm import SVC

dccdataSVMGaussian = SVC(kernel='rbf', class_weight=None, random_state=None)
dccdataSVMGaussian.fit(x_train, y_train)

# Predicting the values

y_pred3 = dccdataSVMGaussian.predict(x_test)

# Confusion matrix and classification report 

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test,y_pred3))
print(classification_report(y_test, y_pred3))

# Evolution metrics

from sklearn import metrics
from sklearn.metrics import roc_curve,roc_auc_score

print('Accuracy: ', (round(metrics.accuracy_score(y_test, y_pred3) * 100, 2)), '%')
print('Precision: ', (round(metrics.precision_score(y_test, y_pred3, average='micro') * 100, 2)), '%')
print('recall_score: ', (round(metrics.recall_score(y_test, y_pred3, average='micro') * 100, 2)), '%')
print('f1_score: ', (round(metrics.f1_score(y_test, y_pred3, average='micro') * 100, 2)), '%')
print('roc_auc_score: ', (round(metrics.roc_auc_score(y_test, y_pred3), 2)))

[[6667  345]
 [1269  699]]
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      7012
           1       0.67      0.36      0.46      1968

    accuracy                           0.82      8980
   macro avg       0.75      0.65      0.68      8980
weighted avg       0.80      0.82      0.80      8980

Accuracy:  82.03 %
Precision:  82.03 %
recall_score:  82.03 %
f1_score:  82.03 %
roc_auc_score:  0.65


In [18]:
# Principal component analysis (PCA) is a statistical technique to convert high dimensional data to low dimensional data
# by selecting the most important features that capture maximum information about the dataset. The features are selected
# on the basis of variance that they cause in the output. The feature that causes highest variance is the first principal
# component. The feature that is responsible for second highest variance is considered the second principal component,
# and so on. It is important to mention that principal components do not have any correlation with each other.

from sklearn.decomposition import PCA

applyPCA = PCA()
#applyPCA = PCA(n_components = 1)

x_train = applyPCA.fit_transform(x_train)
x_test = applyPCA.transform(x_test)
explained_variance = applyPCA.explained_variance_ratio_
print(explained_variance)

[2.64659240e-01 1.20001860e-01 9.53182626e-02 8.71154067e-02
 7.80049594e-02 6.27492061e-02 4.44732502e-02 3.51687421e-02
 2.67278219e-02 2.41659879e-02 1.87921659e-02 1.81322714e-02
 1.41542318e-02 1.35996468e-02 1.26870100e-02 1.09422361e-02
 8.84303795e-03 8.46739881e-03 7.07953386e-03 6.81039101e-03
 4.46725162e-03 4.29955242e-03 3.47868585e-03 3.35240171e-03
 2.78024281e-03 2.72231630e-03 2.43360954e-03 2.36381443e-03
 1.95465164e-03 1.86685601e-03 1.45625787e-03 1.28835213e-03
 1.18291681e-03 1.03241224e-03 9.27759376e-04 9.10164667e-04
 8.20979608e-04 4.98614653e-04 4.12770354e-04 4.05282322e-04
 3.77556927e-04 3.37507982e-04 2.67007600e-04 2.50282301e-04
 2.09004225e-04 2.01070467e-04 1.83338615e-04 1.78677051e-04
 1.53907637e-04 1.38012816e-04 1.35212478e-04 1.28440358e-04
 1.08627960e-04 1.04936110e-04 8.58672568e-05 8.05640038e-05
 7.15239814e-05 6.23809878e-05 5.77417949e-05 4.76808808e-05
 4.16180486e-05 3.07149675e-05 2.77245272e-05 2.72213751e-05
 2.55933406e-05 2.394360

# Random Forest with PCA

In [19]:
# Build Random Forest classification model and Train the model using the training sets

from sklearn.ensemble import RandomForestClassifier

dccdataRF = RandomForestClassifier(bootstrap=True,ccp_alpha=0.0, class_weight=None,
                                  criterion='entropy', max_depth=None, max_features='auto',
                                  max_leaf_nodes=None, max_samples=None,
                                  min_impurity_decrease=0.0, min_impurity_split=None,
                                  min_samples_leaf=1, min_samples_split=2,
                                  n_jobs=None, oob_score=False, random_state=0, verbose=0,
                                  warm_start=False,min_weight_fraction_leaf=0.0, n_estimators=500)

dccdataRF = dccdataRF.fit(x_train, y_train)

# Predict the model with test data set

y1_pred = dccdataRF.predict(x_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y1_pred))
print(classification_report(y_test, y1_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y1_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y1_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y1_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y1_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve 
print('roc_auc_score:', round(roc_auc_score(y_test, y1_pred), 3))

[[6760  252]
 [1397  571]]
              precision    recall  f1-score   support

           0       0.83      0.96      0.89      7012
           1       0.69      0.29      0.41      1968

    accuracy                           0.82      8980
   macro avg       0.76      0.63      0.65      8980
weighted avg       0.80      0.82      0.79      8980

Accuracy: 81.64 %
Precision: 81.64 %
Recall: 81.64 %
f1-score: 81.64 %
roc_auc_score: 0.627


# Decision Tree with PCA

In [20]:
# To build the decision tree model with Over sampling 

from sklearn.tree import DecisionTreeClassifier 

dccdataDT = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                                   max_depth=None, max_features=None, max_leaf_nodes=None,
                                   min_impurity_decrease=0.0, min_impurity_split=None,
                                   min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0,
                                   random_state=None, splitter='best')

dccdataDT = dccdataDT.fit(x_train,y_train)

# Predict with test data

y3_pred = dccdataDT.predict(x_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y3_pred))
print(classification_report(y_test, y3_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y3_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y3_pred) * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y3_pred) * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y3_pred) * 100, 2)), '%')

# Area under ROC curve 
print('roc_auc_score:', round(roc_auc_score(y_test, y3_pred), 3))

[[4361 2651]
 [1125  843]]
              precision    recall  f1-score   support

           0       0.79      0.62      0.70      7012
           1       0.24      0.43      0.31      1968

    accuracy                           0.58      8980
   macro avg       0.52      0.53      0.50      8980
weighted avg       0.67      0.58      0.61      8980

Accuracy: 57.95 %
Precision: 24.13 %
Recall: 42.84 %
f1-score: 30.87 %
roc_auc_score: 0.525


# Logistic Regression with PCA

In [21]:
# To build the 'Logistic Regression' model with random sampling

from sklearn.linear_model import LogisticRegression

dccdataLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                               intercept_scaling=1, max_iter=100, multi_class='auto', 
                               n_jobs=None, penalty='l2', random_state=None,
                               solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

dccdataLR = dccdataLR.fit(x_train,y_train)

# Predict the model with test data set

y5_pred = dccdataLR.predict(x_test)

# Display confusion matrix and classifiction report

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, y5_pred))
print(classification_report(y_test, y5_pred))

# Evaluate the model performance by metrics

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score

# Model Accuracy: how often is the classifier correct?
print("Accuracy:", (round(metrics.accuracy_score(y_test, y5_pred) * 100, 2)), "%")

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", (round(metrics.precision_score(y_test, y5_pred, average='micro') * 100, 2)), '%')

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:", (round(metrics.recall_score(y_test, y5_pred, average='micro') * 100, 2)), "%")

# Model f1-score: weighted average of Precision & Recall
print("f1-score:", (round(metrics.f1_score(y_test, y5_pred, average='micro') * 100, 2)), '%')

# Area under ROC curve 
print('roc_auc_score:', round(roc_auc_score(y_test, y5_pred), 3))

[[6653  359]
 [1256  712]]
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      7012
           1       0.66      0.36      0.47      1968

    accuracy                           0.82      8980
   macro avg       0.75      0.66      0.68      8980
weighted avg       0.80      0.82      0.80      8980

Accuracy: 82.02 %
Precision: 82.02 %
Recall: 82.02 %
f1-score: 82.02 %
roc_auc_score: 0.655


# SVM with PCA

In [22]:
# Training the SVM alforithm

from sklearn.svm import SVC

dccdataSVMGaussian = SVC(kernel='rbf', class_weight=None, random_state=None)
dccdataSVMGaussian.fit(x_train, y_train)

# Predicting the values

y_pred3 = dccdataSVMGaussian.predict(x_test)

# Confusion matrix and classification report 

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test,y_pred3))
print(classification_report(y_test, y_pred3))

# Evolution metrics

from sklearn import metrics
from sklearn.metrics import roc_curve,roc_auc_score

print('Accuracy: ', (round(metrics.accuracy_score(y_test, y_pred3) * 100, 2)), '%')
print('Precision: ', (round(metrics.precision_score(y_test, y_pred3, average='micro') * 100, 2)), '%')
print('recall_score: ', (round(metrics.recall_score(y_test, y_pred3, average='micro') * 100, 2)), '%')
print('f1_score: ', (round(metrics.f1_score(y_test, y_pred3, average='micro') * 100, 2)), '%')
print('roc_auc_score: ', (round(metrics.roc_auc_score(y_test, y_pred3), 2)))

[[6677  335]
 [1281  687]]
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      7012
           1       0.67      0.35      0.46      1968

    accuracy                           0.82      8980
   macro avg       0.76      0.65      0.68      8980
weighted avg       0.80      0.82      0.80      8980

Accuracy:  82.0 %
Precision:  82.0 %
recall_score:  82.0 %
f1_score:  82.0 %
roc_auc_score:  0.65


# KNN with PCA

In [23]:
# Initialize an array that stores the Accuracy

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import sklearn.metrics as metrics
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.metrics import classification_report, confusion_matrix

accuracy = []

for a in range(1, 10, 1):
    k = a
    dccdata1KNN = KNeighborsClassifier(n_neighbors=k)
    dccdata1KNN.fit(x_train, y_train)
    y6_pred = dccdata1KNN.predict(x_test)
    print('KNN_K_value = ', a)
    print("Accuracy:", (round(metrics.accuracy_score(y_test, y6_pred) * 100, 2)), "%")
    print("Precision:", (round(metrics.precision_score(y_test, y6_pred, average='micro') * 100, 2)), '%')
    print("Recall:", (round(metrics.recall_score(y_test, y6_pred, average='micro') * 100, 2)), "%")
    print("f1-score:", (round(metrics.f1_score(y_test, y6_pred, average='micro') * 100, 2)), '%')
    print('roc_auc_score:', round(roc_auc_score(y_test, y6_pred), 3))
    print(confusion_matrix(y_test, y6_pred))
    print('----------------------------------------------------------------------------------------------')

KNN_K_value =  1
Accuracy: 72.22 %
Precision: 72.22 %
Recall: 72.22 %
f1-score: 72.22 %
roc_auc_score: 0.599
[[5739 1273]
 [1222  746]]
----------------------------------------------------------------------------------------------
KNN_K_value =  2
Accuracy: 78.72 %
Precision: 78.72 %
Recall: 78.72 %
f1-score: 78.72 %
roc_auc_score: 0.58
[[6654  358]
 [1553  415]]
----------------------------------------------------------------------------------------------
KNN_K_value =  3
Accuracy: 77.36 %
Precision: 77.36 %
Recall: 77.36 %
f1-score: 77.36 %
roc_auc_score: 0.629
[[6217  795]
 [1238  730]]
----------------------------------------------------------------------------------------------
KNN_K_value =  4
Accuracy: 80.07 %
Precision: 80.07 %
Recall: 80.07 %
f1-score: 80.07 %
roc_auc_score: 0.609
[[6663  349]
 [1441  527]]
----------------------------------------------------------------------------------------------
KNN_K_value =  5
Accuracy: 79.51 %
Precision: 79.51 %
Recall: 79.51 %
f1-scor